# transforms的常见用法


使用Transform主要是在使用其中的各种子类，进行格式转换。

- 输入
- 输出
- 作用



In [5]:
from PIL import Image


In [6]:
pil_img = Image.open('../dataset/furina_logo.jpg')
print(pil_img)


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=599x599 at 0x179AE98E340>


## 使用ToTensor


In [8]:
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

furina_path = '../dataset/furina_logo.jpg'

writer = SummaryWriter('../logs/furina_logs')  # 创建一个SummaryWriter
pil_furina = Image.open(furina_path)  # 读取PIL图片

trans_toTensor = transforms.ToTensor()
tensor_furina = trans_toTensor(pil_furina)  # 转换为tensor格式

writer.add_image('furina_logo', tensor_furina, global_step=0)


## 使用Normalize

Normalize可以对图片进行归一化。

归一化的计算公式可以在函数Normalize的定义中查看，基本上的计算逻辑是：output[channel] = (input[channel] - mean[channel]) / std[channel]


In [9]:
trans_normal = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 定义均值和标准差，这些数据将用于图片归一化计算
img_normal = trans_normal(tensor_furina)  # 对图片进行归一化
writer.add_image('furina_logo', img_normal, global_step=1)  # 将图片写入到新的global_step中。


## 使用Resize


In [12]:
print(pil_img.size)  # 输出原始图像尺寸
trans_resize = transforms.Resize(size=(512, 512))  # 创建resize对象
resized_img = trans_resize(pil_img)  # 进行resize操作
print(f'size after resized: {resized_img}')  # 输出芯图像尺寸

# 注意，该图片现在是PIL格式的图片，所以想要写入到Tensorboard中，还需要转换格式为Tensor
tensor_resized_img = trans_toTensor(resized_img)
writer.add_image('furina_logo', tensor_resized_img, global_step=2)



(599, 599)
size after resized: <PIL.Image.Image image mode=RGB size=512x512 at 0x179AEE4A8E0>


## 使用Compose

该class用于将多种transforms组合到一起，形成一个整体。



In [13]:
"""
这段代码尝试将resize和格式转换（PIL->Tensor）组合到一个Compose中并进行实战"""

trans_resize_2 = transforms.Resize(size=(256,256))
trans_compose = transforms.Compose([
    trans_resize_2,
    trans_toTensor
])  # 先进行一次resize，然后转换成Tensor格式。

Tensor_resized_furina = trans_compose(pil_img)  # 执行compose
writer.add_image('furina_logo', Tensor_resized_furina, global_step=3)  # 将输出结果传递给writer


## 使用RandomCrop随机裁剪

该class用于从原图中随机裁剪一部分图片。


In [17]:
trans_randomCrop = transforms.RandomCrop(128)  # 创建一个RandomCrop

tensor_furina = trans_toTensor(pil_img)  # 先转换成Tensor

for i in range(4,14):
    croped_tensor_furina = trans_randomCrop(tensor_furina)  # 进行随机裁剪
    print(f'img data after croped: {croped_tensor_furina}')  # 输出裁剪图片数据
    writer.add_image('furina_logo', croped_tensor_furina, global_step=i)  # 添加到Tensorboard中


img data after croped: tensor([[[0.7922, 0.7922, 0.7922,  ..., 0.9882, 0.9882, 0.9882],
         [0.7961, 0.7608, 0.8078,  ..., 0.9882, 0.9882, 0.9882],
         [0.3882, 0.4039, 0.3412,  ..., 0.9882, 0.9882, 0.9882],
         ...,
         [1.0000, 0.9686, 1.0000,  ..., 0.9843, 1.0000, 0.9922],
         [1.0000, 0.9961, 0.9843,  ..., 0.9961, 0.9804, 1.0000],
         [1.0000, 0.9725, 0.9843,  ..., 0.9765, 1.0000, 0.9922]],

        [[0.9373, 0.9373, 0.9373,  ..., 0.9804, 0.9804, 0.9804],
         [0.9373, 0.9373, 0.9608,  ..., 0.9804, 0.9804, 0.9804],
         [0.2902, 0.3412, 0.3020,  ..., 0.9804, 0.9804, 0.9804],
         ...,
         [0.9804, 0.9647, 0.9922,  ..., 0.9686, 0.9765, 0.9608],
         [0.9686, 0.9922, 0.9804,  ..., 0.9922, 0.9961, 0.9922],
         [0.9686, 0.9804, 0.9804,  ..., 0.9529, 0.9922, 0.9647]],

        [[0.9725, 0.9725, 0.9725,  ..., 0.9843, 0.9843, 0.9843],
         [1.0000, 0.9098, 0.9412,  ..., 0.9843, 0.9843, 0.9843],
         [0.5490, 0.5961, 0.5882,  